In [1]:
import pandas as pd
import bw2data as bd
import bw2io as bi
import bw2analyzer as ba
import bw2calc as bc
import bw_processing as bwp
from pathlib import Path
import numpy as np
import csv
import sys

In [2]:
bd.projects.set_current('ei38-teaching-25')
ipcc=('IPCC 2013', 'climate change', 'GWP 100a')

In [36]:
activity

'ammonia production, steam reforming, liquid' (kilogram, RER w/o RU, None)

In [3]:
ei = bd.Database('ei 3.8 cutoff')
activity= ei.get('6a7848cd768f5163834eec59c80de49b')
ei_pack=ei.datapackage()

In [4]:
lca1 = bc.LCA({activity: 1}, ipcc)
lca1.lci()
lca1.lcia()
lca1.score

2.477976880419227

In [6]:
heat_acts=[act for act in ei if 'heat production' in  act['name']
#and ('RER' in act['location'] or 'DE' in act['location'] or 'Europe without Switzerland' in act['location'])
#and 'heat' in act['reference product'] 
and 'heat pump' not in act['name']
and 'biomethane' not in act['name']
#and ('natural gas' in act['name'] or 'anthracite' in act['name'])
]

In [7]:
new_heat_supply=[act for act in ei if 'market for electricity, medium voltage' in act['name'] and 'DE' in act['location']][0]
new_heat_supply

'electricity, from municipal waste incineration to generic market for electricity, medium voltage' (kilowatt hour, DE, None)

In [8]:
exchange =list(activity.technosphere())[0]
a= np.array((exchange.input.id, exchange.output.id), bwp.INDICES_DTYPE)

In [9]:
#np.where(ei_pack.data[5] == a)

(array([157541], dtype=int64),)

In [12]:
def recurse_supply(act, depth=0, max_depth=1, flow_dic={},amount=1):
    #global depth
    for exc in act.technosphere():
        
        if exc.input in heat_acts:
            amount = exc['amount']*amount
            flow_dic[exc.input]=amount
            
        elif depth< max_depth:
            depth +=1 
            depth, flow_dic = recurse_supply(exc.input,depth, max_depth, flow_dic, amount)
    
    depth -= 1
    return depth, flow_dic

In [15]:
flow_list=[]
#global depth
#depth=0
#max_depth=3
recurse_supply(activity)

In [16]:
new_flow_list=[]
for exc in flow_list:
    new_flow_list.append((new_heat_supply.id, exc[1]))
    #new_flow_list.append(np.array((new_heat_supply.id, exc['col']), bwp.INDICES_DTYPE))


In [17]:
empty_data= np.zeros(len(flow_list))

In [20]:
data_list=[]
for exc in flow_list:  
    data_list.append(ei_pack.data[6][np.where(ei_pack.data[5] == np.array((exc), bwp.INDICES_DTYPE))][0])

In [21]:
flip=np.zeros(len(data_list)*2, dtype=bool)

In [22]:
indices = np.array(flow_list + new_flow_list, bwp.INDICES_DTYPE)
data = np.array(list(empty_data) + data_list)

In [23]:
dp = bwp.create_datapackage()

In [24]:
dp.add_persistent_vector(
    matrix="technosphere_matrix",
    data_array=data,
    indices_array=indices,
    flip=flip,
    name="new_heat_supply",
)

In [25]:
_, data_objs, _ = bd.prepare_lca_inputs({activity: 1}, ipcc)

In [30]:
lca2 = bc.LCA({activity.id: 1}, data_objs=data_objs+[dp])
lca2.lci()
lca2.lcia()

In [31]:
print(lca1.score, lca2.score)

2.477976880419227 1.782295314458163


In [ ]:
def create_sankey(method):
    i=0
    names=[]
    values=[]
    impact_sankey={}
    while i< len(activities):
        impact_sankey[activities[i][1]]={}
        acti = Solvopet_db.get(activities[i][1])
        if i!=len(activities)-1:
            total_amount= list(Solvopet_db.get(activities[i+1][1]).technosphere())[0]['total']
        else:
            total_amount = func_unit

        lca = bw.LCA({acti:total_amount}, recipe_meth[method])
        lca.lci()
        lca.lcia()
        
        for exc in acti.technosphere():
            lca.redo_lcia({exc.input: exc['amount']*total_amount})
            #impact_sankey[activities[i][1]][exc.input['name']]=lca.score
            names.append(exc.input['name'])
            values.append(lca.score)
            if exc['input'][0]== 'Solvopet db':
                impact_sankey[activities[i][1]]['Thermostat_Flow']={}
                for exc2 in Solvopet_db.get('Thermostat').technosphere():
                    lca.redo_lcia({exc2.input: exc2['amount']*total_amount* exc['amount']})
                    impact_sankey[activities[i][1]]['Thermostat_Flow'][exc2.input['name']]=lca.score
                    names.append(exc2.input['name'])
                    values.append(lca.score)
            #print(exc.input, lca.score)
        i += 1
    lca.redo_lcia({Solvopet_db.get('D700'):func[('Solvopet db', 'D700')] })
    names.append('D700')
    values.append(lca.score)
    return(names, values)

In [ ]:
activities_names, activities_values = create_sankey(method)

In [ ]:
data = dict(
    type='sankey',
    width = 1118,
    height = 772,
    domain = dict(
      x =  [0,1],
      y =  [0,1]
    ),
    textfont = dict(size=12),
    node = dict(
      pad = 15,
      thickness = 40,
      line = dict(
        color = "white",
        width = 0.1
      ),
      label = activities_names,
        
      
      color =   ['mediumaquamarine',
                 'steelblue',
                 'steelblue',
                 'gold',
                 'steelblue',
                 'gold',
                 'gold',
                 'gold',
                 'gold',
                 'mediumaquamarine',
                 'mediumaquamarine',
                 'coral',
                 'gold',
                 'mediumaquamarine',
                 'steelblue',
                 'gold',
                 'gold',
                 'brown',
                 'steelblue',
                 'gold',
                 'gold',
                 'mediumaquamarine',
                 'coral',
                 'gold',
                 'mediumaquamarine',
                 'steelblue',
                 'gold',
                 'mediumaquamarine',
                 'brown',
                 'steelblue',
                 'coral',
                 'gold',
                 'mediumaquamarine',
                 'gold',
                 'steelblue',
                 'white'
                 ]
    ),
    link = dict(
      color = ['mediumturquoise',
                 'lightskyblue',
                 'lightskyblue',
                 'khaki',
                 'lightskyblue',
                 'khaki',
                 'khaki',
                 'khaki',
                 'khaki',
                 'mediumturquoise',
                 'mediumturquoise',
                 'mistyrose',
                 'khaki',
                 'mediumturquoise',
                 'lightskyblue',
                 'khaki',
                 'khaki',
                 'lightcoral',
                 'lightskyblue',
                 'khaki',
                 'khaki',
                 'mediumturquoise',
                 'mistyrose',
                 'khaki',
                 'mediumturquoise',
                 'lightskyblue',
                 'khaki',
                 'mediumturquoise',
                 'lightcoral',
                 'lightskyblue',
                 'mistyrose',
                 'khaki',
                 'mediumturquoise',
                 'khaki',
                 'lightskyblue',
                 'lightskyblue'
                 ],
      source = [0,1,2,3,4 , 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34],
      target = [4,4,4,4,14,14,14,14,14,14,14,14,11,11,18,18,18,18,25,25,25,25,25,22,22,29,29,29,29,34,34,30,30,34,35],
      value = activities_values
  ))

layout =  dict(
    title = method,
    font = dict(
      size = 10
    )
)

fig = dict(data=[data], layout=layout)
plotly.offline.plot(fig, filename= 'sankey_'+method+'oeko'+'.html', validate=False)
#py.iplot(fig, validate=False)
